In [3]:
#load modules
from Bio import Entrez, Seq, SeqIO, pairwise2, Align
import numpy as np

#load nucleotide sequences from NCBI 
def get_seq(db, key, rettype="gb"):
    Entrez.email = "A.N.Other@example.com"
    handle = Entrez.efetch(db=db, id=key, rettype=rettype)
    return SeqIO.read(handle, "gb")

## Call NCBI API

In [4]:
#[welche daten werden geladen?]
#hvr-1
nea_1a = get_seq("nucleotide", "AF011222")
nea_1b = get_seq("nucleotide", "AF282971")

#[welche daten werden geladen?]
#hvr-2
nea_2a = get_seq("nucleotide", "AF142095")
nea_2b = get_seq("nucleotide", "AF282972")

In [27]:
n = 206 #known length

nea_1 = nea_1a.seq + nea_1b.seq
nea_2 = nea_2a.seq + nea_2b.seq

## Local Alignments

In [33]:
score = np.zeros(n)
startat = np.zeros(n)


aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.substitution_matrix = MatrixInfo.nuc44
with open("d_loop.fasta", "r") as handle:
    sequences = []
    d_loop = SeqIO.parse(handle, "fasta")
    """for seq in d_loop:
        a = aligner.align(nea_1, seq.seq)
        for i in a:"""
    for seq in d_loop:
        sequences.append(seq.seq)
    sequences.append(nea_1)
    sequences.append(nea_2)
            
        
    
        
        
            
            
"""
from Bio import AlignIO
handle = ...
for a in AlignIO.parse(handle, "fasta-m10"):
  assert len(a) == 2, "Should be pairwise!"
  print("Alignment length %i" % a.get_alignment_length())
  for record in a:
    print("%s %s %s" % (record.seq, record.name, record.id))
-----------------------------------------------------------------
>>> from Bio import Align
>>> aligner = Align.PairwiseAligner()
>>> alignments = aligner.align("ACCGT", "ACG")
>>> for alignment in sorted(alignments):
...     print("Score = %.1f:" % alignment.score)
...     print(alignment)"""

NameError: name 'MatrixInfo' is not defined

In [24]:
print(len(nea_1))

736


In [41]:
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO

with open("d_loop.fasta", "r") as handle:
    sequences = []
    d_loop = SeqIO.parse(handle, "fasta")
    """for seq in d_loop:
        a = aligner.align(nea_1, seq.seq)
        for i in a:"""
    for seq in d_loop:
        sequences.append(SeqRecord(seq.seq))
    sequences.append(SeqRecord(nea_1))
    
al1 = AlignIO.read(open("d_loop.fasta", "r"), "fasta")  
print(al1)


SingleLetterAlphabet() alignment with 206 rows and 1600 columns
GTAGCACACAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Andalusia(AF382011).txt
GTAGCACTCAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AF346963).txt
GTAGCACTACAGAATAATTCCCCTTGCACACCCGATAAATCCGA...AAT Aust(AF346964).txt
GTAGCACTACAGATAAATTCCCCTTGCACACCCGATAAATCCGA...AAT Aust(AF346965).txt
GTAGCACACAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289051).txt
GTAGCACTACAAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAT Aust(AY289052).txt
GTAGCACTACGAATAAATTCCCCTTGCACACCCGATAAATCCAA...GAA Aust(AY289053).txt
GTAGCACTAAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289054).txt
GTAGCACTAAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289055).txt
GTAGCACTAAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289056).txt
GTAGCACTAAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289057).txt
GTAGCACTAAGAATAAATTCCCCTTGCACACCCGATAAATCCGA...GAA Aust(AY289058).txt
GTAGCACTACAGAAAAATTCCCCTTGCACACCCGATAAATCCGA...ATC Aust(AY289059).txt
GTAGCACTAAGAATAAATTCC